In [1]:
import chess
import numpy as np
import pandas as pd
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [5]:
def fen_to_vector(fen):
    board = chess.Board(fen)
    vector = np.zeros(64)
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            vector[square] = {
                "P": 1, "N": 2, "B": 3, "R": 4, "Q": 5, "K": 6,
                "p": -1, "n": -2, "b": -3, "r": -4, "q": -5, "k": -6
            }[piece.symbol()]
    return vector

In [22]:
# Charger les données depuis un CSV
def load_data(file_path, max_lines=None):
    data = pd.read_csv(file_path)
    if max_lines:
        data = data.head(max_lines)
    X = []
    y = []
    for i, row in data.iterrows():
        try:
            fen = row['fen']
            best_move = row['analysed_best_move']
            X.append((fen))
            y.append((best_move))
        except Exception as e:
            print(f"Erreur avec la ligne {i}: {e}")
    return np.array(X), np.array(y)


In [23]:
# Charger les données à partir du fichier CSV
file_path = 'data\data_cleaned.csv'  # Remplacez par le chemin de votre fichier CSV
data = pd.read_csv(file_path)

# Extraire les colonnes 'fen' et 'best_move'
X = data['fen']  # Variable explicative
y = data['most_popular_move_of_the_category']  # Variable à expliquer

In [26]:
# Diviser les données en ensembles d'entraînement et de validation
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

ImportError: cannot import name '_spbase' from 'scipy.sparse._base' (c:\Users\theol\anaconda3\lib\site-packages\scipy\sparse\_base.py)

In [ ]:
# Construire le modèle
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, input_dim=64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(64 * 64, activation='softmax')  # Sortie pour 4096 mouvements possibles
])

In [ ]:
# Compilation
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Entraînement
history = model.fit(X, y, validation_split=0.2, epochs=10, batch_size=512)

In [ ]:
# Sauvegarder le modèle
model.save("chess_ai_model.h5")

In [ ]:
# Charger le modèle
model = tf.keras.models.load_model("chess_ai_model.h5")

# Tester une prédiction
fen = "1B1b4/8/2pP2p1/5p2/k7/8/5PK1/8 b - -"
fen_vector = fen_to_vector(fen)
prediction = model.predict(np.array([fen_vector]))
predicted_move_index = np.argmax(prediction)

# Convertir l'indice en mouvement (par exemple, 0 -> 'a1a2')
from_square = predicted_move_index // 64
to_square = predicted_move_index % 64
predicted_move = chess.SQUARE_NAMES[from_square] + chess.SQUARE_NAMES[to_square]

print(f"Mouvement prédit : {predicted_move}")